In [ ]:
!pip install -q torch transformers huggingface_hub datasets wandb

In [ ]:
!pip install -q transformers[torch]

In [1]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM
import wandb

2024-06-20 08:15:09.714945: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 08:15:09.715036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 08:15:09.964776: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Connect to WANDB to store logs

In [2]:
from getpass import getpass
api_token = getpass("Enter your Wandb api key: ")

Enter your Wandb api key:  ········································


In [3]:
wandb.login(key=api_token)
wandb.init(project='Viveka')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nadikapoudel16 (grievance). Use `wandb login --relogin` to force relogin


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
df = pd.read_csv("/kaggle/input/medsimplify/medSimplify_data.csv")


In [6]:
df.isnull().sum()

medical    0
simple     0
dtype: int64

In [7]:
df.shape

(11374, 2)

In [8]:
model_name = "luqh/ClinicalT5-base"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [10]:
# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True, max_length=512)

In [11]:
# # Define the target tokenization
# def tokenize_target(examples):
#     return tokenizer(examples['simple'], padding="max_length", truncation=True, max_length=512)

In [13]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [14]:
# Apply the tokenization to the 'medical' and 'simple' columns for training data
train_encodings = tokenizer(list(train_df['medical']), truncation=True, padding=True, max_length=512)
train_labels = tokenizer(list(train_df['simple']), truncation=True, padding=True, max_length=512)

# Apply the tokenization to the 'medical' and 'simple' columns for validation data
val_encodings = tokenizer(list(val_df['medical']), truncation=True, padding=True, max_length=512)
val_labels = tokenizer(list(val_df['simple']), truncation=True, padding=True, max_length=512)


In [15]:
class MedicalDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

In [16]:

train_dataset = MedicalDataset(train_encodings, train_labels)
val_dataset = MedicalDataset(val_encodings, val_labels)


In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="medical_jargons_simplifierT5",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    warmup_steps=50,
    save_steps = 500,
    eval_steps=500,
    logging_steps=500,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    load_best_model_at_end=True,     # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Metric to optimize over
    logging_dir="./logs",
    report_to = "wandb",
    push_to_hub=True,

)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
model = T5ForConditionalGeneration.from_pretrained(model_name, from_flax=True)


All Flax model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the Flax model and are newly initialized: ['lm_head.weight', 'encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

In [24]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
500,3.336900,0.569968
1000,0.601000,0.527919
1500,0.572000,0.513735
2000,0.555600,0.505140
2500,0.513200,0.499052
3000,0.540600,0.494073
3500,0.513000,0.490894
4000,0.532800,0.488018
4500,0.530400,0.484577
5000,0.521500,0.482500


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=11375, training_loss=0.6428621504228194, metrics={'train_runtime': 14104.3703, 'train_samples_per_second': 3.226, 'train_steps_per_second': 0.806, 'total_flos': 2.77045392310272e+16, 'train_loss': 0.6428621504228194, 'epoch': 5.0})

In [25]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/nadika/medical_jargons_simplifierT5/commit/6e5d0126db3ea8de3cb45c23cf4fd59dce599bb0', commit_message='End of training', commit_description='', oid='6e5d0126db3ea8de3cb45c23cf4fd59dce599bb0', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
# Load the fine-tuned model and tokenizer
model_path = "nadika/medical_jargons_simplifierT5"  
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)





config.json:   0%|          | 0.00/810 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [27]:
# Function to simplify medical jargon using the model
def simplify_medical_jargon(medical_text):
    input_text = f"simplify: {medical_text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs.input_ids, max_length=1080, num_beams=4, early_stopping=True)
    simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return simplified_text

In [ ]:
# Example input
medical_text = "Prescribed oral anticoagulants for atrial fibrillation management."

# Get the simplified explanation
simplified_text = simplify_medical_jargon(medical_text)
print(f"Original: {medical_text}")
print(f"Simplified: {simplified_text}")
